In [34]:
#Random forest algorithm for predicting election results
#Mike Pachnik

from csv import reader
from math import sqrt
from random import seed
from random import randrange



# Load the data
def load_csv(filename):
	data = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			data.append(row)
	return data

# prepare the data with a k folds split
def k_folds_split(data, n_folds):
	data_split = list()
	data_copy = list(data)
	fold_size = int(len(data) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(data_copy))
			fold.append(data_copy.pop(index))
		data_split.append(fold)
	return data_split

# Evaluate the algorithm
def evaluate(data, n_folds, *args):
	folds = k_folds_split(data, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = random_forest(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = measure_accuracy(actual, predicted)
		scores.append(accuracy)
	return scores

# Calculate accuracy 
def measure_accuracy(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Calculate gini index
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Split a dataset according to an attribute
def test_split(index, value, data):
	left = list()
	right = list()
	for row in data:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def terminal(group):
	outcome = [row[-1] for row in group]
	return max(set(outcome), key=outcome.count)

# Child splits
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	#check no split
	if not left or not right:
		node['left'] = node['right'] = terminal(left + right)
		return
	#check max depth
	if depth >= max_depth:
		node['left'], node['right'] = terminal(left), terminal(right)
		return
	#left child
	if len(left) <= min_size:
		node['left'] = terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	#right child
	if len(right) <= min_size:
		node['right'] = terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

# Create a random subsample 
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

# Make a prediction from bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)




In [35]:
#load data
filename = 'Forest_Data.csv'
dataset = load_csv(filename) 

#set parameters

#ratio of total data to be sampled
sample_ratio = .25
n_folds = 2
max_depth = 2
min_size = 1
#number of attributes to be considered for each tree
n_features = int(sqrt(len(dataset[0])-1))

#run algorithm for 1, 5, and 10 trees
#Warning: algorithm may take upwards of 30-45 minutes depending on input parameters

#minimum parameters have been set as default for demonstration purposes
#and do not reflect full accuracy of the algorithm 
for n_trees in [1, 5, 10]:
	scores = evaluate(dataset, n_folds, max_depth, min_size, sample_ratio, n_trees, n_features)
	print(n_trees)
	print(scores)
	print('Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

1
[84.45859872611466, 87.89808917197452]
Accuracy: 86.178%


KeyboardInterrupt: 